In [129]:
import time
import requests
import urllib.request

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs

In [130]:
###############
## 기본정보 세팅후 네이버 로그인하기

options = Options()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_experimental_option('excludeSwitches', ['enable-logging'])
driver = webdriver.Chrome('./chromedriver.exe', chrome_options=options)
driver.implicitly_wait(5)
driver.get('https://nid.naver.com/nidlogin.login')

C:\Users\Administrator\AppData\Local\Temp\ipykernel_11908\3117264847.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe', chrome_options=options)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_11908\3117264847.py:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('./chromedriver.exe', chrome_options=options)


In [131]:
###############
## 기본정보 세팅후 강남구청 로그인하기

아동이름 = '서진영'
아동생년월일 = '2019-01-17'
휴대폰번호 = '010-6685-0501'
인원수 = '1'
거주지역 = '장지동' 

#/resv/apply/segok_indoor_playground/list.do?sch_week_day=2022-10&mid=ID04_04070903

## 도곡놀이터
#PLAYGROUND = ['https://www.gangnam.go.kr/resv/apply/memewe_clean_playground/list.do?mid=ID04_02071902'
#             ,]

## 세곡놀이터
PLAYGROUND = ['https://www.gangnam.go.kr/resv/apply/segok_indoor_playground/list.do?mid=ID04_04070903'
              ,'https://www.gangnam.go.kr/resv/reqst/segok_indoor_playground//insert.do?']

## 일원놀이터
#PLAYGROUND = ['https://www.gangnam.go.kr/resv/apply/irwon_playground/list.do?mid=ID04_04074703']  

PLAY = 'playground/'
INSERT = '/insert.do?'
LOGIN = 'https://www.gangnam.go.kr/login.do?mid=ID91_9101&returnUrl='+PLAYGROUND[0][PLAYGROUND[0].index('go.kr')+5:]
driver.get(LOGIN)

In [132]:
###############
## 로그인한 정보로 예약 목록 불러오기 : 11시 정각에 실행해야 할것

headers = {
"User-Agent":
    "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36"
}
ss = requests.session()
ss.headers.update(headers)

for cookie in driver.get_cookies():
    c = {cookie['name']: cookie['value']}
    ss.cookies.update(c)

res = ss.get(PLAYGROUND[0])
reserve = bs(res.text, "html.parser")
for item in reserve.find_all("td"):
    if item.find('div', 'fc-sat') != None or item.find('div', 'holiday-color') != None:
        #주말
        #print(item.select_one('span').get_text(), '일')    
        #for link in item.select('a'):
        #    print(link.get_text().strip(), link['href'][link['href'].find('playground/')+len('playground/'):link['href'].find('/add.do')])
        pass
    else:
        #평일
        print(item.select_one('span').get_text(), '일 ***** ', end= '')
        for link in item.select('a'):
            print(link.get_text().strip(), end=' 예약번호::::[')
            print(link['href'][link['href'].find('playground/')+len('playground/'):link['href'].find('/add.do')], end=']:::::========== ')
        print('')

31 일 ***** 
1 일 ***** 
2 일 ***** 
3 일 ***** 
4 일 ***** 
7 일 ***** 
8 일 ***** 
9 일 ***** 16:00 (11 / 20) 예약번호::::[1509]:::::========== 
10 일 ***** 13:30 (6 / 20) 예약번호::::[1510]:::::========== 16:00 (13 / 20) 예약번호::::[1511]:::::========== 
11 일 ***** 
14 일 ***** 
15 일 ***** 16:00 (17 / 20) 예약번호::::[1519]:::::========== 
16 일 ***** 16:00 (14 / 20) 예약번호::::[1520]:::::========== 
17 일 ***** 16:00 (10 / 20) 예약번호::::[1522]:::::========== 
18 일 ***** 16:00 (18 / 20) 예약번호::::[1523]:::::========== 
21 일 ***** 
22 일 ***** 
23 일 ***** 
24 일 ***** 
25 일 ***** 
28 일 ***** 
29 일 ***** 
30 일 ***** 
1 일 ***** 
2 일 ***** 


In [133]:
###################################################
#상위 목록에서 예약번호를 복사해서 값 변경후 실행  #
#######                                     #######
예약번호를넣어주세요 = '1522'               #######
#######                                     #######
###################################################

REQ_STR = []
REQ_STR.extend(PLAYGROUND[1][:PLAYGROUND[1].index(PLAY)+len(PLAY)] + 예약번호를넣어주세요 + PLAYGROUND[1][PLAYGROUND[1].index(INSERT):])
REQ_STR.extend(PLAYGROUND[0][PLAYGROUND[0].index('mid='):])
REQ_STR.extend('&rar_user_name='+아동이름+'&rar_hp_no='+휴대폰번호+'&rar_person_cnt='+인원수)
REQ_STR.extend('&rar_optn_keys=1&rar_optn_values='+아동생년월일+'&rar_optn_keys=2&rar_optn_values='+거주지역)
REQ_STR.extend('&rar_reqst_key=0&rar_file_key=&rar_sms_yn=Y&rar_agree_yn=Y&rar_terms_yn=Y')

result = ss.post(''.join(REQ_STR))

In [134]:
res_list = ss.post(PLAYGROUND[0].replace('apply', 'reqst'))
my_list = bs(res_list.text, "html.parser")
idx = 0
for item in my_list.find_all("td"):
    if idx == 1:
        print('예약일자 : ', item.text)
    if idx == 2:
        print('예약아동 : ', item.text.strip())
        #print(item['href'])
    if idx == 3:
        print('연락처 : ', item.text)
    if idx == 4:
        print('신청인원 : ', item.text)
    idx = idx+1

예약일자 :  2022-11-17 16:00
예약아동 :  서진영
연락처 :  010-6685-0501
신청인원 :  1


In [46]:
#headers = {
#    "User-Agent":"Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36",
#    'Host': 'www.gangnam.go.kr',
#    'Origin': 'https://www.gangnam.go.kr',
#    'Referer': 'https://www.gangnam.go.kr/resv/reqst/segok_indoor_playground/1522/add.do?mid=ID04_04070903'
#
#}

insert_payload = {
    'mid': 'ID04_04070903',
    'rar_reqst_key': '0',
    'rar_file_key': '',
    'rar_user_name': '서진영',
    'rar_hp_no': '010-6685-0501',
    'rar_person_cnt': '1',
    'rar_optn_keys': '1',
    'rar_optn_values': '2021-01-17',
    'rar_optn_keys': '2',
    'rar_optn_values': '장지동',
    'rar_sms_yn': 'Y',
    'rar_agree_yn': 'Y',
    'rar_terms_yn': 'Y '   
}
res_reserve = ss.post('https://www.gangnam.go.kr/resv/reqst/segok_indoor_playground/'+예약번호+'/insert.do?', data=insert_payload)